<a href="https://colab.research.google.com/github/aetev/Hearth-Stone-Python-Simulator/blob/main/card_enc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [73]:
import json
import tensorflow as tf
import re
from tensorflow.keras.layers import Embedding
# Assuming AtomicCards.json is in your Google Drive, adjust the path if necessary
file_path = '/content/drive/MyDrive/MTGdata/AtomicCards.json'


with open(file_path, 'r') as f:
  data = json.load(f)['data']

In [6]:
def filter_legal_cards(data):
    legal_commander_data = {}
    for card_name, card_data in data.items():
        try:
            if card_data[0]['legalities']['commander'] == 'Legal':
                legal_commander_data[card_name] = card_data[0] # Store only the first element of card_data
        except (KeyError, IndexError):
            pass  # Or print a message, log it, etc.
    return legal_commander_data

filtered_data = filter_legal_cards(data)


In [35]:
items = list(filtered_data.items())

second_key, second_value = items[11]

text = second_value['text']
print(text)

Up to two target creatures each get +1/+0 and gain first strike until end of turn. (They deal combat damage before creatures without first strike.)
Madness {1}{R} (If you discard this card, discard it into exile. When you do, cast it for its madness cost or put it into your graveyard.)


In [103]:
class TextCleaningLayer(tf.keras.layers.Layer):
    def __init__(self,sentence_list, **kwargs):
        super(TextCleaningLayer, self).__init__(**kwargs)
        self.vectorizer = TextVectorization(standardize=None, ngrams=2, output_mode='int',output_sequence_length=150)
        self.vectorizer.adapt(cleaned_text(sentence_list))
        self.embedding_layer = Embedding(input_dim=len(self.vectorizer.get_vocabulary()), output_dim=128)

    def clean_text(self, text):
        # Convert input to string tensor if it's not already
        if not isinstance(text, tf.Tensor):
            text = tf.convert_to_tensor(text, dtype=tf.string)

        cleaned_text = tf.strings.lower(text)
        cleaned_text = tf.strings.regex_replace(cleaned_text, '\n', '')
        cleaned_text = tf.strings.regex_replace(cleaned_text, '\.', ' ')
        cleaned_text = tf.strings.regex_replace(cleaned_text, '\{|\}', ' ')
        cleaned_text = tf.strings.regex_replace(cleaned_text, '\([^)]*\)', '')
        cleaned_text = tf.strings.regex_replace(cleaned_text, '[^\w\s+/\-]', '')
        cleaned_text = tf.strings.regex_replace(cleaned_text, '\s+', ' ')
        return cleaned_text

    def call(self, inputs):
        cleaned_text = self.clean_text(inputs)
        vectorized_text = self.vectorizer(cleaned_text)
        embedded_text = self.embedding_layer(vectorized_text)

        return embedded_text

# Example usage with a batch of texts
batch_texts = tf.constant([text, "hello world test", "Another example"])
cleaned_text_layer = TextCleaningLayer(batch_texts)
cleaned_batch = cleaned_text_layer.call(batch_texts)
cleaned_batch


<tf.Tensor: shape=(3, 150, 128), dtype=float32, numpy=
array([[[ 3.5406042e-02, -2.7813781e-02, -5.3073093e-04, ...,
         -2.2590792e-02, -2.7713621e-02,  4.8941541e-02],
        [-4.2807292e-02,  3.9193779e-04,  4.4053499e-02, ...,
         -2.5137460e-02,  3.8147084e-03, -1.1839330e-02],
        [ 1.4596548e-02, -7.8021772e-03, -2.0301118e-03, ...,
         -3.8030647e-02, -4.2037688e-02, -4.6179544e-02],
        ...,
        [-4.7132254e-02,  3.9570834e-02,  1.6621780e-02, ...,
         -2.2411956e-02, -2.6728138e-03,  3.0308042e-02],
        [-4.7132254e-02,  3.9570834e-02,  1.6621780e-02, ...,
         -2.2411956e-02, -2.6728138e-03,  3.0308042e-02],
        [-4.7132254e-02,  3.9570834e-02,  1.6621780e-02, ...,
         -2.2411956e-02, -2.6728138e-03,  3.0308042e-02]],

       [[ 3.4413222e-02, -1.7720807e-02, -2.7569044e-02, ...,
          1.3886716e-02, -4.5064654e-02, -1.9868959e-02],
        [ 2.8596666e-02, -3.4812190e-02,  2.9844914e-02, ...,
          9.9298954e-03, -1.

In [95]:


# Assuming 'vectorized_text' is defined from the previous code

# Define the embedding dimension
embedding_dim = 128  # You can adjust this value

# Create the embedding layer
embedding_layer = Embedding(input_dim=len(vectorizer.get_vocabulary()), output_dim=embedding_dim)

# Embed the vectorized text
embedded_text = embedding_layer(vectorized_text)

# Print the shape of the embedded text
print("Shape of embedded text:", embedded_text.shape)

# Now 'embedded_text' contains the embedded word sequences.
embedded_text


Shape of embedded text: (100, 128)


<tf.Tensor: shape=(100, 128), dtype=float32, numpy=
array([[ 0.01018262, -0.01596836,  0.02729158, ...,  0.0019261 ,
        -0.03301797,  0.03238631],
       [ 0.00254347,  0.03750212, -0.02892951, ...,  0.02927576,
        -0.00974404, -0.02733778],
       [ 0.02289246, -0.02318466, -0.04464887, ..., -0.03656035,
        -0.02206706,  0.03524668],
       ...,
       [-0.02593027,  0.04201361,  0.02043838, ..., -0.03076535,
         0.01708222, -0.0017042 ],
       [-0.02593027,  0.04201361,  0.02043838, ..., -0.03076535,
         0.01708222, -0.0017042 ],
       [-0.02593027,  0.04201361,  0.02043838, ..., -0.03076535,
         0.01708222, -0.0017042 ]], dtype=float32)>